<a href="https://colab.research.google.com/github/miha1D/mlops-exercise/blob/main/Image_Prediction_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os
import numpy as np
import shutil
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import tensorflow as tf

In [ ]:
# Mount Google Drive
try:
    drive.mount('/content/drive', force_remount=True)
    print("Drive mounted successfully")
except ValueError as e:
    print("Error mounting Drive:", e)

Mounted at /content/drive
Drive mounted successfully


In [ ]:
# Define base directory
base_dir = '/content/drive/MyDrive/New Project/dataset/'

In [ ]:
# Function to move images to a new directory
def move_images(src_dir, dest_dir, class_names, num_to_move):
    for class_name in class_names:
        class_src_dir = os.path.join(src_dir, class_name)
        class_dest_dir = os.path.join(dest_dir, class_name)

        if not os.path.exists(class_dest_dir):
            os.makedirs(class_dest_dir)

        images = os.listdir(class_src_dir)
        images_to_move = images[:num_to_move]

        for img in images_to_move:
            shutil.move(os.path.join(class_src_dir, img), os.path.join(class_dest_dir, img))
            print(f"Moved {img} from {class_src_dir} to {class_dest_dir}")

In [ ]:
# Verify the structure and images in each category
def verify_structure(base_dir):
    for category in os.listdir(base_dir):
        category_path = os.path.join(base_dir, category)
        if os.path.isdir(category_path):
            print(f"Category: {category}")
            print(f"Number of images: {len(os.listdir(category_path))}")
            for img_file in os.listdir(category_path)[:5]:
                print(f" - {img_file}")

Category: Park
Number of images: 0
Category: Cars
Number of images: 0
Category: animals
Number of images: 0
Category: train
Number of images: 3
 - Park
 - Cars
 - animals
Category: validation
Number of images: 3
 - Park
 - Cars
 - animals


In [ ]:
# Initial verification of structure
verify_structure(base_dir)

In [ ]:
# Create new directories for training and validation
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

if not os.path.exists(train_dir):
    os.makedirs(train_dir)
if not os.path.exists(validation_dir):
    os.makedirs(validation_dir)

In [ ]:
# Define class names and move images
class_names = ['Park', 'Cars', 'animals']

Found 20 images belonging to 5 classes.
Found 4 images belonging to 5 classes.
Number of training samples: 20
Number of validation samples: 4


In [ ]:
# Move images to validation directory first
move_images(base_dir, validation_dir, class_names, 1)

In [ ]:
# Move remaining images to train directory
for class_name in class_names:
    class_src_dir = os.path.join(base_dir, class_name)
    remaining_images = len(os.listdir(class_src_dir)) - 1
    move_images(base_dir, train_dir, [class_name], remaining_images)

In [ ]:
# Verify the structure for training and validation
print("\nTraining Directory Structure:")
verify_structure(train_dir)
print("\nValidation Directory Structure:")
verify_structure(validation_dir)


In [ ]:
# Image Data Generator for Training and Validation
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

def create_generators(base_dir, batch_size):
    train_generator = datagen.flow_from_directory(
        base_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )

    validation_generator = datagen.flow_from_directory(
        base_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )
    return train_generator, validation_generator

In [ ]:
# Update the data generators with the new directory structure
train_generator, validation_generator = create_generators(train_dir, batch_size=2)
print("Number of training samples:", train_generator.samples)
print("Number of validation samples:", validation_generator.samples)

In [ ]:
# Load the pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

model.save('/content/drive/MyDrive/New Project/my_model.h5')

In [ ]:
# Re-evaluate the model only if there are samples in the validation set
if validation_generator.samples > 0:
    val_loss, val_accuracy = model.evaluate(validation_generator)
    print(f"Validation Loss: {val_loss}")
    print(f"Validation Accuracy: {val_accuracy}")
else:
    print("No samples in the validation set.")

In [ ]:
# Function to load and preprocess image
def load_and_preprocess_image(img_path):
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

In [ ]:
# Example usage for new image prediction
img_path = '/content/drive/MyDrive/New Project/New Images Prediction/Cars/Cars1.jpg'
img_array = load_and_preprocess_image(img_path)

In [ ]:
# Function to make and display predictions
def predict_and_display(new_images_dir, model, class_labels):
    for category in os.listdir(new_images_dir):
        category_path = os.path.join(new_images_dir, category)
        if os.path.isdir(category_path):
            for img_file in os.listdir(category_path):
                img_path = os.path.join(category_path, img_file)
                img_array = load_and_preprocess_image(img_path)

                predictions = model.predict(img_array)
                predicted_class = np.argmax(predictions, axis=1)
                predicted_label = class_labels[predicted_class[0]]
                print(f"Image: {img_file} - Predicted label: {predicted_label}")

                img = load_img(img_path)
                plt.imshow(img)
                plt.title(f"Predicted label: {predicted_label}")
                plt.show()

In [ ]:
# Predict and display for new images
new_images_dir = '/content/drive/MyDrive/New Project/New Images Prediction/'
predict_and_display(new_images_dir, model, list(train_generator.class_indices.keys()))